In [2]:
import os
import random
import time
import json
import warnings 
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from tqdm import tqdm
from pycocotools.coco import COCO
import cv2
import albumentations as A

# 시각화를 위한 라이브러리
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from matplotlib.patches import Patch
import sys
sys.path.append('../smp/')
from data_loader import *
from utils import *
from tqdm import tqdm

In [3]:
dataset_path  = '/opt/ml/input/data'
anns_file_path = dataset_path + '/' + 'train_all.json'
config_path = '../smp/config.json'
global_config = load_json(config_path)
train_transform = A.Compose([
                        # ToTensorV2()
                        ])
train_dataset = BaseDataset(dataset_path=dataset_path, mode='train', transform=train_transform, global_config = global_config)

loading annotations into memory...
Done (t=4.62s)
creating index...
index created!


In [4]:
hists = []
for idx in tqdm(range(len(train_dataset))):
    mask = np.array(train_dataset[idx][1], np.uint8)
    hist = cv2.calcHist([mask],[0], mask=None, histSize=[11], ranges=[0,11])
    hist = hist.squeeze()
    hists.append(hist)

100%|██████████| 2617/2617 [00:51<00:00, 50.67it/s]


In [5]:
hist_array = np.array(hists, np.float32)
hist_all = hist_array.sum(axis=0)
total = hist_all.sum()
print(type(hist_all))
print(type(total))
print(hist_all)
print(total)
hist_ratio = hist_all/total
with np.printoptions(precision=2, suppress=True):
    print(1./hist_ratio)

<class 'numpy.ndarray'>
<class 'numpy.float32'>
[4.7788720e+08 1.5475035e+07 6.2123852e+07 4.9005330e+06 6.1616410e+06
 5.2640800e+06 1.9775412e+07 1.0351425e+07 8.0124008e+07 3.1722700e+05
 3.6506180e+06]
686031100.0
[   1.44   44.33   11.04  139.99  111.34  130.32   34.69   66.27    8.56
 2162.59  187.92]


In [12]:
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

alpha = (1./hist_ratio)
alpha = alpha / np.max(alpha)
with np.printoptions(precision=4, suppress=True):
    print(repr(alpha))


array([0.0007, 0.0205, 0.0051, 0.0647, 0.0515, 0.0603, 0.016 , 0.0306,
       0.004 , 1.    , 0.0869], dtype=float32)
